In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
def load_data(directory, label, img_size=(32, 32)):
    data = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
        img = cv2.resize(img, img_size)  
        data.append(img)
        labels.append(label)
    return np.array(data), np.array(labels)

In [3]:
closed_eye_path = r"C:\Users\sathy\DBA\model\Data\Closed"
open_eye_path = r"C:\Users\sathy\DBA\model\Data\Open"

In [4]:
open_eyes, open_labels = load_data(open_eye_path, label=0)
closed_eyes, closed_labels = load_data(closed_eye_path, label=1)

In [5]:
X = np.vstack((open_eyes, closed_eyes))
y = np.hstack((open_labels, closed_labels))

In [6]:
X = X / 255.0 
X = X.reshape(-1, 32, 32, 1)  
y = to_categorical(y, 2) 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from tensorflow.keras import layers, models

def build_lenet():
    model = models.Sequential([
        layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)),
        layers.AveragePooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
        layers.AveragePooling2D(pool_size=(2, 2)),  
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(2, activation='softmax')  
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


model = build_lenet()


c:\Users\sathy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.fit(X_train, y_train, epochs=11, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9723 - loss: 0.0510 - val_accuracy: 0.9656 - val_loss: 0.1215
Epoch 2/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9908 - loss: 0.0438 - val_accuracy: 0.9588 - val_loss: 0.1322
Epoch 3/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9912 - loss: 0.0317 - val_accuracy: 0.9691 - val_loss: 0.1110
Epoch 4/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9836 - loss: 0.0464 - val_accuracy: 0.9519 - val_loss: 0.1766
Epoch 5/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9793 - loss: 0.0394 - val_accuracy: 0.9588 - val_loss: 0.1297
Epoch 6/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9917 - loss: 0.0228 - val_accuracy: 0.9588 - val_loss: 0.1437
Epoch 7/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9928 - loss: 0.0172 - val_accuracy: 0.9759 - val_loss: 0.1612
Epoch 8/11
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9918 - loss: 0.0202 - val_accuracy: 0.9656 - val_loss

In [13]:
model.save("drowsiness_lenet_model.h5")

In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9568 - loss: 0.1842 
Test Accuracy: 0.9519
